In [1]:
import pyspark
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 12:38:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import date_format
from pyspark.sql.functions import to_date

23/11/20 12:38:57 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
schema = StructType([ 
    StructField("ad_id", IntegerType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("action", StringType(), True)
])
data = [
    {"ad_id": 1, "user_id": 1, "action": "Clicked"},
    {"ad_id": 2, "user_id": 2, "action": "Clicked"},
    {"ad_id": 3, "user_id": 3, "action": "Viewed"},
    {"ad_id": 5, "user_id": 5, "action": "Ignored"},
    {"ad_id": 1, "user_id": 7, "action": "Ignored"},
    {"ad_id": 2, "user_id": 7, "action": "Viewed"},
    {"ad_id": 3, "user_id": 5, "action": "Clicked"},
    {"ad_id": 1, "user_id": 4, "action": "Viewed"},
    {"ad_id": 2, "user_id": 11, "action": "Viewed"},
    {"ad_id": 1, "user_id": 2, "action": "Clicked"}
]
# Convert the list of Row objects into a DataFrame
df = spark.createDataFrame(data, schema)

df.show()
print(df.dtypes)

+-----+-------+-------+
|ad_id|user_id| action|
+-----+-------+-------+
|    1|      1|Clicked|
|    2|      2|Clicked|
|    3|      3| Viewed|
|    5|      5|Ignored|
|    1|      7|Ignored|
|    2|      7| Viewed|
|    3|      5|Clicked|
|    1|      4| Viewed|
|    2|     11| Viewed|
|    1|      2|Clicked|
+-----+-------+-------+

[('ad_id', 'int'), ('user_id', 'int'), ('action', 'string')]


In [8]:
df.createOrReplaceTempView("Ads")

In [31]:
result = spark.sql(
    """
    SELECT ad_id,
        FORMAT_NUMBER(IFNULL(ROUND(100 * SUM(CASE WHEN action = 'Clicked' THEN 1 ELSE 0 END)
        /SUM(CASE WHEN action = 'Ignored' THEN 0 ELSE 1 END), 2), 0), 2) AS ctr
    FROM Ads
    GROUP BY ad_id
    ORDER BY ctr desc, ad_id
    """
)
result.show()

+-----+-----+
|ad_id|  ctr|
+-----+-----+
|    1|66.67|
|    3|50.00|
|    2|33.33|
|    5| 0.00|
+-----+-----+



In [29]:
result = spark.sql(
    """
    SELECT sum(1)
    FROM Ads
    WHERE action = 'Clicked'
    
    """
)
result.show()

+------+
|sum(1)|
+------+
|     4|
+------+

